## Reed-Solomon code

In [142]:
class RSCode:
    
    def __init__(self, n, k, q, alpha = None):
        
        if not (k < n and n <= q):
            raise ValueError('Invalid values for n, k, and q.')
            
        self.p0, self.m = is_prime_power(q, get_data = True)
            
        self.n = n
        self.k = k
        self.q = q
        
        self.d = self.n - self.k + 1
        
        self.tau = floor((self.n-self.k)/2)
        
        # Initializing field
        self.F = GF(self.q)
        self.R = PolynomialRing(self.F, 'X')
        self.p = self.F.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n)])
        else:
            self.alpha = alpha
        
        # Constructing generator matrix
        self.G = matrix(self.F, k, n, lambda i,j : self.alpha[j]**i)
        
    def Encoding(self, m, zeropad = True):
        
        m = self._IntToPol(m)
        
        rem = len(m) % self.k
        
        if rem != 0:
            if zeropad:
                m.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(m), self.k):
            c.extend(self.EncodeChunk(m[i:i+self.k]))
        
        
        #print(c)
        c = self._PolToInt(c)
        #print(c)
        c = self._IntToByteString(c)
        #print(c)
        
        return c
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G
        return c
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(r),self.n):
            c.extend(self.BivariateInterpolation(r[i:i+self.n]))
            
        return c
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.F, self.n, self.tau + self.k, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.F, self.n, self.tau + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau+self.k]))
        Q1 = self.R(list(sol[self.tau+self.k:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.F(0)]*(self.k-len(out)))

        return out
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q:
                raise ValueError('Invalid symbol')
            m_out.append(self.F(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)

    def _IntToByteString(self, int_array):

        if(self.q != 2^4):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of integers less than 256 to binary representation

        if any([(item > 15 or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return list(''.join([format(item, '04b') for item in int_array]))

In [143]:
C = RSCode(15,7,2**4)
G = C.G
latex(transpose(G))

\left(\begin{array}{rrrrrrr}
1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & z_{4} & z_{4}^{2} & z_{4}^{3} & z_{4} + 1 & z_{4}^{2} + z_{4} & z_{4}^{3} + z_{4}^{2} \\
1 & z_{4}^{2} & z_{4} + 1 & z_{4}^{3} + z_{4}^{2} & z_{4}^{2} + 1 & z_{4}^{2} + z_{4} + 1 & z_{4}^{3} + z_{4}^{2} + z_{4} + 1 \\
1 & z_{4}^{3} & z_{4}^{3} + z_{4}^{2} & z_{4}^{3} + z_{4} & z_{4}^{3} + z_{4}^{2} + z_{4} + 1 & 1 & z_{4}^{3} \\
1 & z_{4} + 1 & z_{4}^{2} + 1 & z_{4}^{3} + z_{4}^{2} + z_{4} + 1 & z_{4} & z_{4}^{2} + z_{4} & z_{4}^{3} + z_{4} \\
1 & z_{4}^{2} + z_{4} & z_{4}^{2} + z_{4} + 1 & 1 & z_{4}^{2} + z_{4} & z_{4}^{2} + z_{4} + 1 & 1 \\
1 & z_{4}^{3} + z_{4}^{2} & z_{4}^{3} + z_{4}^{2} + z_{4} + 1 & z_{4}^{3} & z_{4}^{3} + z_{4} & 1 & z_{4}^{3} + z_{4}^{2} \\
1 & z_{4}^{3} + z_{4} + 1 & z_{4}^{3} + 1 & z_{4}^{3} + z_{4}^{2} & z_{4}^{3} + z_{4}^{2} + 1 & z_{4}^{2} + z_{4} & z_{4}^{3} + z_{4}^{2} + z_{4} + 1 \\
1 & z_{4}^{2} + 1 & z_{4} & z_{4}^{3} + z_{4} & z_{4}^{2} & z_{4}^{2} + z_{4} + 1 & z_{4}^{3} \\
1 & z_{4}^{3} 

In [153]:
m = [1,2,3,4,5,6,7]
print(C._IntToByteString(m))
c = C.Encoding(m)
RM = ReedMullerCode(1,3)
test = vector(GF(2), [0,1,0,1])
cc = RM.Encoding(c)
cc

['0', '0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '1', '1', '1']


(0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0)

In [119]:
0, 1, 0, 1, 1, 0, 1, 0)

16

In [129]:
cc = []

for i in range(15):
    current_word = []
    for i in range(i,len(c),15):
        print(c[i].list())
        current_word.append(c[i])
    
    #print(vector(GF(2), current_word))
    print(current_word.polynomial_coeffs())

AttributeError: 'sage.rings.finite_rings.element_givaro.FiniteField_givaroElement' object has no attribute 'list'

In [96]:
bin(1)

'0b1'

In [113]:
class ReedMullerCode:
    
    def __init__(self, r, m):
            
        self.r = r
        self.m = m
        self.q = 2
        
        self.n = 2**self.m
        self.k = sum(binomial(self.m,i) for i in range(self.r + 1))
        self.d = 2**(self.m - self.r)
        
        # Initializing field
        self.F = FiniteField(self.q)
        
        self.G = matrix(self.F, self.k, self.n, lambda i,j : 0)
        
        #print('RM({0},{1}) has the parameters:'.format(self.r, self.m))
        #print('n, k, d:')
        #print(self.n,self.k,self.d)
        self.G = self.get_G(self.r, self.m)
        #print('and generator matrix:')
        #print(self.G)
        
    
    def get_G(self, r, m):
        n = 2**m
        k = sum(binomial(m,i) for i in range(r + 1))
        d = 2**(m - r)

        #if (n == k):
            # trivial code

        if (n == d):
            # repetition code
            return matrix(self.F, k, n, lambda i,j : 1)

        elif (d == 2 and n == k + 1):
            # parity check code
            PCC = codes.ParityCheckCode(GF(self.q), k)
            return PCC.generator_matrix()

        else:
            return block_matrix([ [self.get_G(r,m-1), self.get_G(r,m-1)], [0, self.get_G(r-1,m-1)] ])
    
    
    def Encoding(self, message, zeropad = True):
        
        rem = len(message) % self.k
        
        if rem != 0:
            if zeropad:
                message.extend([self.F(0)]*(self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c = []
        
        # Encoding each chunk of size k
        for i in range(0, len(message), self.k):
            c.extend(self.EncodeChunk(message[i:i+self.k]))
        
        return vector(self.F, c)
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G

        return c
        
    
    def Decoding(self, received):
        
        # Check input size
        if len(received) % self.n != 0:
            raise ValueError('Invalid input size')
            
        c = []
        
        for i in range(0,len(received),self.n):
            c.extend(self.DecodeChunk(received[i:i+self.n], self.r, self.m))
            
        return vector(self.F, c)
                
            
    def DecodeChunk(self, word, r, m):
        
        word = vector(self.F, word)
        
        if (self.n % len(word)) != 0:
            raise ValueError('Invalid chunk size')
        
        if (r == 0):
            return self.repetition_decode(word)
        
        if (m == r + 1):
            return 'parity check'
        
        l = len(word)
        u1 = word[:l//2]
        u2_v = word[l//2:]
        
        v_error = u1 + u2_v

        v = self.DecodeChunk(v_error, r - 1, m - 1)
        u2 = u2_v + v
        
        u1_decoded = self.DecodeChunk(u1, r, m - 1)
        u2_decoded = self.DecodeChunk(u2, r, m - 1)
        u = 0
        
        if (u1_decoded == 'parity check' and u2_decoded == 'parity check'):
            u = self.parity_check_compare(u1, u2)
            return vector(self.F, list(u) + list(u + v))
        else:
            return vector(self.F, list(u1_decoded) + list(u2_decoded + v))
        
    
    
    def repetition_decode(self, word):
        
        if word.hamming_weight() >= len(word.list())//2:
            return vector(self.F, [1 for i in range(len(word.list()))])
        elif word.hamming_weight() < len(word.list())//2:
            return vector(self.F, [0 for i in range(len(word.list()))])
        else:
            return "decoding failure"
        
    def parity_check_compare(self, word1, word2):
        decoded1 = word1[:-1]
        decoded2 = word2[:-1]
        
        if (((decoded1.hamming_weight() % 2) == 0 and word1[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word1[-1] == 1)):
            return word1
        elif (((decoded2.hamming_weight() % 2) == 0 and word2[-1] == 0) or ((decoded1.hamming_weight() % 2) == 1 and word2[-1] == 1)):
            return word2
        else:
            return "decoding failure"
        